# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# importing necessary csv files

In [2]:
df_ratings = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/ratings.csv")
df_movies = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/movies.csv", encoding  = 'unicode_escape') #contains the title
df_tags= pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/tags.csv") 
df_genome_scores = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/genome-scores.csv") 
df_links = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/links.csv")
df_genome_tags = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/genome-tags.csv")

# Checking each of them

### df_movies

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### df_links

In [4]:
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### df_ratings

In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
df_ratings.shape

(25000095, 4)

In [7]:
df_ratings['userId'].value_counts().shape

(162541,)

#### out of 25000095 ratings done, 162541 are the unique raters. this is till a big number, so now we will narrow our search by removing all those users, who hadn't rated at least 500 movies

In [8]:
x = df_ratings['userId'].value_counts() > 500

In [9]:
y = x[x].index

In [10]:
y.shape

(9663,)

In [11]:
rating = df_ratings[df_ratings['userId'].isin(y)]

In [12]:
rating.shape

(8935061, 4)

In [13]:
rating

,userId,movieId,rating,timestamp
254,3,1,4.0,1439472215
255,3,29,4.5,1484754967
256,3,32,4.5,1439474635
257,3,50,5.0,1439474391
258,3,111,4.0,1484753849
...,...,...,...,...
24996936,162519,4748,2.0,1000947348
24996937,162519,4789,1.0,1000946127
24996938,162519,4855,5.0,1001884147
24996939,162519,4857,5.0,1001886067


### now merging df_movies and rating

In [14]:
movie_details = df_movies.merge(rating, on = 'movieId')

In [15]:
movie_details.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,1167582601
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,956264593
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,942014736
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,1074594380


In [16]:
movie_details.shape

(8935061, 6)

#### lets remove timestamp column as we don't need it

In [17]:
movie_details.drop(columns=['timestamp'], inplace = True)

In [18]:
movie_details.shape

(8935061, 5)

In [19]:
movie_details.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


#### getting the info that which movie is rated how many number of times

In [20]:
number_rating = movie_details.groupby('title')['rating'].count().reset_index()

In [21]:
number_rating.rename(columns = {'rating' : 'number of rating'}, inplace = True)

In [22]:
number_rating.head()

,title,number of rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",42
2,#1 Cheerleader Camp (2010),7
3,#Captured (2017),2
4,#Female Pleasure (2018),2


### Now merging movie_details and number_rating

In [23]:
df_tillnow = movie_details.merge(number_rating, on = 'title')

In [24]:
df_tillnow.shape

(8935061, 6)

In [25]:
df_tillnow.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,8078
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,8078
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,8078
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,8078
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,8078


#### by setting the threshold of 50 in number_of_rating so as to more narrow down our search space

In [26]:
df_tillnow1 = df_tillnow[df_tillnow['number of rating']>=50] #taking only the valuable one

In [27]:
df_tillnow1.shape

(8596657, 6)

In [28]:
df_tillnow1.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,8078
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,8078
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,8078
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,8078
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,8078


#### now we don't need number_of_rating column

In [29]:
df_tillnow1.drop(columns=['number of rating'], inplace = True)

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
df_tillnow1.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


In [31]:
df_tillnow1['rating'] = df_tillnow1['rating'].astype(float)

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_tillnow1.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


In [33]:
movie_pivot = df_tillnow1.pivot_table(columns='title', index = 'userId', values = 'rating')

In [34]:
movie_pivot

title,'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: Return of Xander Cage (2017),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
72,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162495,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
162508,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
movie_pivot.shape

(9663, 11716)

In [36]:
movie_pivot.fillna(0, inplace = True)

In [37]:
movie_pivot

title,'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: Return of Xander Cage (2017),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
72,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162495,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
162508,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
from scipy.sparse import csr_matrix

In [39]:
movie_sparse = csr_matrix(movie_pivot)

In [40]:
movie_sparse

<9663x11716 sparse matrix of type '<class 'numpy.float64'>'
	with 8596205 stored elements in Compressed Sparse Row format>

In [41]:
from sklearn.neighbors import NearestNeighbors

In [42]:
modelr = NearestNeighbors(n_neighbors = 11, algorithm = 'brute', metric = 'euclidean')
modelr.fit(movie_sparse)
def recommend(user):
    #lets get the similar users
    distances, similar_users = modelr.kneighbors(movie_pivot.loc[user].values.reshape(1, -1))
    #making the urecommend(3)ser's row as an dataframe itself for proper use
    userdf = movie_pivot.loc[user].to_frame()
    userdf.columns = ['rating'] #there are two columns, title as index, rating the 2nd column
    """print('userdf\n')
    print(userdf.head())
    print('\n') """
    
    #now getting those movies which are rated by the by the user itself, as we don't want to suggest it again
    #X = userdf['rating'].value_counts() > 1
    #Y = X[X]
    rated_by_user = userdf[userdf['rating']>1]
    #min-max normaliztion
    normalized_rated_by_user = (rated_by_user - rated_by_user.min())/(rated_by_user.max() - rated_by_user.min())
    print('rated_by_user\n')
    print(normalized_rated_by_user.head(10))
    print('\n')
    
    #getting the index of rated_by_user as a list, as it contains the name of the movies which user had already watched
    #watched_already = rated_by_user.index.values.tolist()
    watched_already = normalized_rated_by_user.index.values.tolist()
    #print("\n", len(watched_already))
    
    #now combining all the similar users rating, by making the index common
    #by using loop, we are gonna form new dataframe and keep adding them
    #1st making a new dataframe, with already 1st user in it
    
    other_user_df = movie_pivot.iloc[similar_users[0][1]].to_frame()
    other_user_df.columns = ['rating'] #there are two columns, title as index, rating the 2nd column
    for i in range(2,len(similar_users)):
        subdf = movie_pivot.iloc[similar_users[0][i]].to_frame()
        subdf.columns = ['rating']
        other_user_df = other_user_df.add(subdf, fill_value = 0)
    
    
    #there still be many movies which all the rest of the users wouldn't have watched as well
    #lets get rid of them
    #Xo = other_user_df['rating'].value_counts() > 1
    #Yo = Xo[Xo]
    rated_by_other_users = other_user_df[other_user_df['rating']>1]
    """print('rated_by_other_users\n')
    print(rated_by_other_users.head(10))
    print('\n')"""
    
    #now that we have filtered dataframe where the movies wathed by others is there,
    #we have to get rid of those movies, which the original user had already watched
    #here, watched_already list will help us out so as to this 
    
    filtered_rated_by_other_users = rated_by_other_users[~rated_by_other_users.index.isin(watched_already)]
    #print("\n", filtered_rated_by_other_users.shape[0])
    """print('filtered_rated_by_other_users\n')
    print(filtered_rated_by_other_users.head(10))
    print('\n')"""
    
    #now as we have a list of unwatched movies by our given users,
    #now we are gonna normalize our ratings so that the ratings is b/w 0-1
    
    #filtered_rated_by_other_users['rating'] = filtered_rated_by_other_users['rating'].div(len(similar_users[0]) - 1)
    
    #min-max normaliztion
    filtered_rated_by_other_users = (filtered_rated_by_other_users - filtered_rated_by_other_users.min())/(filtered_rated_by_other_users.max() - filtered_rated_by_other_users.min())                                                          
    
    #now we are gonna sort the values in descending order so that we can get the highest rated movies combined by the other users
    recommendation_to_user = filtered_rated_by_other_users.sort_values('rating', ascending = False)
    print('recommendation_to_user\n')
    print(recommendation_to_user.head(10))

In [43]:
recommend(3)

rated_by_user

                                                      rating
title                                                       
102 Dalmatians (2000)                               0.000000
15 Minutes (2001)                                   0.500000
2 Fast 2 Furious (Fast and the Furious 2, The) ...  0.666667
2001: A Space Odyssey (1968)                        1.000000
2012 (2009)                                         0.166667
22 Jump Street (2014)                               0.333333
25th Hour (2002)                                    0.833333
27 Dresses (2008)                                   0.333333
28 Days (2000)                                      0.333333
28 Days Later (2002)                                0.333333


recommendation_to_user

                                                      rating
title                                                       
Spirited Away (Sen to Chihiro no kamikakushi) (...  1.000000
Planet Earth II (2016)                      

In [44]:
recommend(72)

rated_by_user

                                                      rating
title                                                       
'burbs, The (1989)                                  0.333333
10 Things I Hate About You (1999)                   1.000000
101 Dalmatians (One Hundred and One Dalmatians)...  0.333333
12 Angry Men (1957)                                 1.000000
200 Cigarettes (1999)                               0.333333
2001: A Space Odyssey (1968)                        0.333333
2010: The Year We Make Contact (1984)               0.666667
28 Days (2000)                                      0.666667
8MM (1999)                                          0.333333
Abyss, The (1989)                                   0.666667


recommendation_to_user

                                              rating
title                                               
Peeping Tom (1960)                               1.0
To Kill a Mockingbird (1962)                     1.0
Kids (1995)    